In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip3 install transformers
!pip3 install hazm
!pip3 install advertools
!pip3 install parsivar

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 30.0 MB/s 
     |████████████████████████████████| 7.6 MB 51.1 MB/s 
     |████████████████████████████████| 163 kB 73.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 316 kB 24.1 MB/s 
     |████████████████████████████████| 233 kB 71.3 MB/s 
     |████████████████████████████████| 1.4 MB 49.8 MB/s 
  Created wheel for nltk: filename=nltk-3.3-py3-none-any.whl size=1394488 sha256=e037f3d711c0ae19b2319a0c646c8941bc13b7927a8af4d26ca33a890f0a6c02
  Stored in directory: /root/.cache/pip/wheels/9b/fd/0c/d92302c876e5de87ebd7fc0979d82edb93e2d8d768bf71fac4
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp37-cp37m-linux_x86_64.whl size=154936 sha256=8bca10247d4ea7c1192a95e4beaac003f71d190612e0511d5a8bbf6ea25c5a5b
  Stored in directory: /root/.cac

In [3]:
import numpy as np
import pandas as pd
import torch
from hazm import *
import advertools as adv
from parsivar import Tokenizer
from transformers import BertTokenizer
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from transformers import BertModel, BertForSequenceClassification
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [5]:
train = pd.read_csv("/content/drive/My Drive/Colab Notebooks/study plan/Projects/Project 1/data/Train-word.csv", sep='\t')
val = pd.read_csv("/content/drive/My Drive/Colab Notebooks/study plan/Projects/Project 1/data/Val-word.csv", sep='\t')
test = pd.read_csv("/content/drive/My Drive/Colab Notebooks/study plan/Projects/Project 1/data/Test-word.csv", sep='\t')

In [6]:
print('Training set Information')
print(train.info(), "\n")
print('missing values')
print(train.isnull().sum(), "\n")
print('duplicated rows')
print(train.duplicated().sum(), "\n")
data = train.drop_duplicates(keep='first')
print('train shape')
print(train.shape)

Training set Information
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7266 entries, 0 to 7265
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   premise     7266 non-null   object
 1   hypothesis  7266 non-null   object
 2   label       7266 non-null   object
dtypes: object(3)
memory usage: 170.4+ KB
None 

missing values
premise       0
hypothesis    0
label         0
dtype: int64 

duplicated rows
0 

train shape
(7266, 3)


In [7]:
print('Validation set Information')
print(val.info(), "\n")
print('missing values')
print(val.isnull().sum(), "\n")
print('duplicated rows')
print(val.duplicated().sum(), "\n")
data = val.drop_duplicates(keep='first')
print('validation shape')
print(val.shape)

Validation set Information
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1537 entries, 0 to 1536
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   premise     1537 non-null   object
 1   hypothesis  1537 non-null   object
 2   label       1537 non-null   object
dtypes: object(3)
memory usage: 36.1+ KB
None 

missing values
premise       0
hypothesis    0
label         0
dtype: int64 

duplicated rows
0 

validation shape
(1537, 3)


In [8]:
print('Test set Information')
print(test.info(), "\n")
print('missing values')
print(test.isnull().sum(), "\n")
print('duplicated rows')
print(test.duplicated().sum(), "\n")
data = test.drop_duplicates(keep='first')
print('test shape')
print(test.shape)

Test set Information
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1564 entries, 0 to 1563
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   premise           1564 non-null   object
 1   hypothesis        1564 non-null   object
 2   label             1564 non-null   object
 3   hard(hypothesis)  1564 non-null   int64 
 4   hard(overlap)     1564 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 61.2+ KB
None 

missing values
premise             0
hypothesis          0
label               0
hard(hypothesis)    0
hard(overlap)       0
dtype: int64 

duplicated rows
0 

test shape
(1564, 5)


In [11]:
train.head(3)

,premise,hypothesis,label
0,اولین انتقال و نفوذ طبیعی فرهنگ و تمدن اسلامی ...,نخستین انتقال و نفوذ طبیعی فرهنگ و تمدن اسلامی...,e
1,اولین انتقال و نفوذ طبیعی فرهنگ و تمدن اسلامی ...,کانون های جغرافیایی مصر، اندلس و شام، نخستین ر...,c
2,اولین انتقال و نفوذ طبیعی فرهنگ و تمدن اسلامی ...,سیسیل بعد از اسپانیا بزرگ ترین کانونی بود که ه...,n


In [12]:
test.head(3)

,premise,hypothesis,label,hard(hypothesis),hard(overlap)
0,دوران امامت امام صادق علیه السلام، مصادف است ب...,امام سجاد (ع) در دورانی امامت کردند که همزمان ...,c,0,1
1,دوران امامت امام صادق علیه السلام، مصادف است ب...,دستگاه فاسد حکومتی با صرف هزینه های هنگفت، سعی...,n,1,0
2,با شهادت امام رضا(ع) مرحله جدیدی از تلاش ائمه ...,دوران محنت اهل بیت پس از شهادت امام رضا(ع) آغا...,e,0,0


In [13]:
print("Train label: \n", train['label'].value_counts(), "\n")
print("Validation label: \n", val['label'].value_counts(), "\n")
print("Test label: \n", test['label'].value_counts())

Train label: 
 n    2448
e    2429
c    2389
Name: label, dtype: int64 

Validation label: 
 n    523
e    515
c    499
Name: label, dtype: int64 

Test label: 
 n    535
e    519
c    510
Name: label, dtype: int64


In [14]:
LABELS_TO_IDS = {label: ids for ids, label in enumerate(list(set(train['label'].tolist())))}
IDS_TO_LABELS = {ids: label for label, ids in LABELS_TO_IDS.items()}

In [15]:
print(LABELS_TO_IDS)
print(IDS_TO_LABELS)

{'c': 0, 'n': 1, 'e': 2}
{0: 'c', 1: 'n', 2: 'e'}


##Preprocessing

In [16]:
class Preprocessing():

  def __init__(self, normalizer, tokenizer, lemmatizer, stopwords:list, punc:list):
    """
        Class initializer
    Args:
        stopwords(list): list of stop words
        punc(list): list of punctuations
    Returns:
    """
    self.normalizer = normalizer
    self.tokenizer = tokenizer
    self.lemmatizer = lemmatizer
    self.stopwords = stopwords
    self.punc = punc

  def normalize(self,X:str) -> str:
    """
        Normalizing
    Args:
        X(str): input string
    Return:
        X(str): string normalized
    """
    return self.normalizer.normalize(X)

  def tokenize(self,X:str) -> list:
    """
        Tokenization
    Args:
        X(str): input string
    Returns:
        tokens(list): list of words
    """
    return self.tokenizer.tokenize_words(X)
  
  def lema(self,X:list) -> list:
    """
        Lemmatization
    Args:
        X(list): list of words
    Returns:
        X(list): list of words lemmatized
    """
    return [self.lemmatizer.lemmatize(word) for word in X]

  def stopword_removal(self,X:list) -> list:
    """
        Stopword removal
    Args:
        X(list): list of words
    Returns:
        X(list): list of words that stopwords removed
    """
    return [token for token in X if token[0] not in self.stopwords]
  
  def join_sents(self,X:list) -> str:
    """
        Sentences creation
    Args:
        X(list): list of words
    Returns:
        X(str): sentence created
    """
    sents = ' '.join([token for token in X])
    return sents.strip()

  def punc_removal(self,X:str) -> str:
    """
        Punctuation removal
    Args:
        X(str): input string
    Returns:
        X(str): string that punctuations removed
    """
    for ele in X:
        if ele in self.punc:
            X = X.replace(ele, "")
    return X 
  
  def preprocessor(self,X:str) -> str:
    """
         Data Preprocessor
    Args:
        X(str): input string
    Return:
        X(str): preprocessed string
    """
    #Normalizing
    X = self.normalize(X)
    #Tokenization
    X = self.tokenize(X)
    #Lemmatization
    X = self.lema(X)
    #Stopword_removal
    X = self.stopword_removal(X)
    #Sentences creation
    X = self.join_sents(X)
    #Punchuation removal
    X = self.punc_removal(X)
    return X

In [17]:
normalizer = Normalizer()
tokenizer = Tokenizer()
lemmatizer = Lemmatizer()
stopwords = sorted(adv.stopwords['persian'])
punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~1234567890؟۱۲۳۴۵۶۷۸۹۰''' 

In [18]:
Preprocess = Preprocessing(normalizer, tokenizer, lemmatizer, stopwords, punc)

In [19]:
train['premise'] = [Preprocess.preprocessor(text) for text in train['premise']]
train['hypothesis'] = [Preprocess.preprocessor(text) for text in train['hypothesis']]

In [20]:
val['premise'] = [Preprocess.preprocessor(text) for text in val['premise']]
val['hypothesis'] = [Preprocess.preprocessor(text) for text in val['hypothesis']]

In [21]:
test['premise'] = [Preprocess.preprocessor(text) for text in test['premise']]
test['hypothesis'] = [Preprocess.preprocessor(text) for text in test['hypothesis']]

##Tokenization & Dataset & DataLoader

In [22]:
tokenizer = BertTokenizer.from_pretrained("HooshvareLab/bert-base-parsbert-uncased")

Downloading:   0%|          | 0.00/1.22M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/434 [00:00<?, ?B/s]

In [23]:
max_len = 128
train_batch_size = 32
val_batch_size = 32
test_batch_size = 32

In [24]:
class CustomDataset(Dataset):
    def __init__(self, data, tokenizer, max_len, labels_to_ids):
        self.len = len(data)
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.labels_to_ids = labels_to_ids
       
    def __len__(self):
        return self.len
             
    def __getitem__(self, index):
        text1 = ' '.join(self.data['premise'].tolist()[index].split())
        text2 = ' '.join(self.data['hypothesis'].tolist()[index].split())
        label = self.data['label'].tolist()[index]
        inputs = self.tokenizer.encode_plus(text1,
                                            text2,
                                            max_length=self.max_len,
                                            padding='max_length',
                                            return_token_type_ids=True,
                                            truncation=True)
        label = self.labels_to_ids[label]
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        return {
             'ids': torch.tensor(ids, dtype=torch.long),
             'mask': torch.tensor(mask, dtype=torch.long),
             'targets': torch.tensor(label, dtype=torch.long)
        }

In [25]:
train_dataset = CustomDataset(train, tokenizer, max_len, LABELS_TO_IDS)

In [26]:
val_dataset = CustomDataset(val, tokenizer, max_len, LABELS_TO_IDS)

In [27]:
test_dataset = CustomDataset(test, tokenizer, max_len, LABELS_TO_IDS)

In [28]:
TRAIN_LOADER = DataLoader(train_dataset, batch_size=train_batch_size)

In [29]:
VAL_LOADER = DataLoader(val_dataset, batch_size=val_batch_size)

In [30]:
TEST_LOADER = DataLoader(test_dataset, batch_size=test_batch_size)

In [31]:
print(len(TRAIN_LOADER))
print(len(VAL_LOADER))
print(len(TEST_LOADER))

228
49
49


##Finetuning

In [32]:
model = BertForSequenceClassification.from_pretrained("HooshvareLab/bert-fa-base-uncased", num_labels=len(LABELS_TO_IDS))

Downloading:   0%|          | 0.00/440 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/654M [00:00<?, ?B/s]

Some weights of the model checkpoint at HooshvareLab/bert-fa-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification w

In [33]:
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(100000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [34]:
epochs = 10
learning_rate = 1e-5
max_grad_norm = 1e-5

loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [35]:
def calcuate_accu(big_idx, targets):
    n_correct = (big_idx == targets).sum().item()
    return n_correct

In [ ]:
def train(epoch, model, loader, optimizer, loss_function, device):
    tr_loss = 0
    n_correct = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    y_predict, y_true = [], []
    model.train()
    #with tqdm(loader, unit='batch') as tepoch:
    for _, data in enumerate(tqdm(loader), 0):
        # for _, data in tqdm(enumerate(loader, 0)):
        ids = data['ids'].to(device, dtype=torch.long)
        mask = data['mask'].to(device, dtype=torch.long)
        targets = data['targets'].to(device, dtype=torch.long)
        
        outputs = model(ids, mask)
        #outputs = torch.tensor(outputs)
        
        loss = loss_function(outputs.logits, targets)
        tr_loss += loss.item()
        big_val, big_idx = torch.max(outputs.logits, dim=1)
        n_correct += calcuate_accu(big_idx, targets)
        for y_pred in big_idx.cpu().numpy():
             y_predict.append(y_pred)
        for y_target in targets.cpu().numpy():
             y_true.append(y_target)
        nb_tr_steps += 1
        nb_tr_examples += targets.size(0)
        if _ % 1000 == 0:
            loss_step = tr_loss / nb_tr_steps
            accu_step = (n_correct * 100) / nb_tr_examples
            print(f"\nTraining Loss per 1000 steps: {loss_step}")
            f1 = f1_score(y_true, y_predict, average='macro')
            print(f"Training F1-Score per 1000 steps: {f1}")
            # print(f"Training Accuracy per 1000 steps: {accu_step}")
            print("--------------------------------------------------")
        optimizer.zero_grad()
        loss.backward()
        # # When using GPU
        optimizer.step()

    print(f'The Total Accuracy for Epoch {epoch}: {(n_correct * 100) / nb_tr_examples}')
    epoch_loss = tr_loss / nb_tr_steps
    epoch_accu = (n_correct * 100) / nb_tr_examples
    print(f"Training Loss Epoch {epoch}: {epoch_loss}")
    f1 = f1_score(y_true, y_predict, average='macro')
    print(f"Training F1-Score {epoch}: {f1}")
    # print(f"Training Accuracy Epoch {epoch}: {epoch_accu}")


In [ ]:
def valid(model, loader, device, dataset_type):
    nb_tr_steps = 0
    nb_tr_examples = 0
    n_correct = 0
    y_predict = []
    y_true = []
    model.eval()
    with torch.no_grad():
        for _, data in enumerate(tqdm(loader), 0):
            ids = data['ids'].to(device, dtype=torch.long)
            mask = data['mask'].to(device, dtype=torch.long)
            targets = data['targets'].to(device, dtype=torch.long)
            outputs = model(ids, mask)
            # try:
            big_val, big_idx = torch.max(outputs.logits, dim=1)
            # except:
            #     print("\n##########################################")
            #     print(f"Data at: {_}")
            #     print(outputs)
            #     print(outputs.data)
            #     print(outputs.data.shape)
            #     print("##########################################")
            #     continue
            n_correct += calcuate_accu(big_idx, targets)
            for y_pred in big_idx.cpu().numpy():
                y_predict.append(y_pred)
            for y_target in targets.cpu().numpy():
                y_true.append(y_target)
            nb_tr_steps += 1
            nb_tr_examples += targets.size(0)
            if _ % 1000 == 0:
                accu_step = (n_correct * 100) / nb_tr_examples
                f1 = f1_score(y_true, y_predict, average='macro')
                print(f"\n{dataset_type} F1-Score {epoch}: {f1}")
                # print(f"{dataset_type} Accuracy per 1000 steps: {accu_step}")
                print("-------------------------------------")
    # epoch_accu = (n_correct * 100) / nb_tr_examples
    print(f'The Total Accuracy for Epoch {epoch}: {(n_correct * 100) / nb_tr_examples}')
    epoch_accu = (n_correct * 100) / nb_tr_examples
    f1 = f1_score(y_true, y_predict, average='macro')
    print(f"{dataset_type} F1-Score {epoch}: {f1}")
    # print(f"Training Accuracy Epoch {epoch}: {epoch_accu}")
    print("========================================================")
    return y_true, y_predict

In [ ]:
def evaluation_method(y_true, y_pred, IDS_TO_LABELS):
    f1 = f1_score(y_true, y_pred, average='macro')
    acc = accuracy_score(y_true, y_pred)
    pre = precision_score(y_true, y_pred, average='macro')
    rec = recall_score(y_true, y_pred, average='macro')

    clf_report = classification_report(y_true, y_pred)
    return {
        "y-true": [IDS_TO_LABELS[y] for y in y_true],
        "y-pred": [IDS_TO_LABELS[y] for y in y_pred],
        "f1": f1, "accuracy": acc,
        "precision": pre, "recall": rec,
        "clf-report": clf_report
    }

In [ ]:
for epoch in range(epochs):
  print(f"Training epoch: {epoch + 1}")
  train(epoch, model, TRAIN_LOADER, optimizer, loss_fn, device)
  print("=================================================================")

Training epoch: 1


  0%|          | 0/228 [00:00<?, ?it/s]


Training Loss per 1000 steps: 1.1162911653518677
Training F1-Score per 1000 steps: 0.24725274725274726
--------------------------------------------------


 72%|███████▏  | 164/228 [01:51<00:44,  1.44it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
 97%|█████████▋| 221/228 [02:31<00:04,  1.41it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed

The Total Accuracy for Epoch 0: 54.39031103770988
Training Loss Epoch 0: 0.9090867387621027
Training F1-Score 0: 0.5295236024582315
Training epoch: 2


...

The Total Accuracy for Epoch 8: 89.6091384530691
Training Loss Epoch 8: 0.26680640581374365
Training F1-Score 8: 0.8955882448603391
Training epoch: 10


  0%|          | 0/228 [00:00<?, ?it/s]


Training Loss per 1000 steps: 0.16702842712402344
Training F1-Score per 1000 steps: 0.9388888888888888
--------------------------------------------------


 72%|███████▏  | 164/228 [01:56<00:45,  1.42it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
 97%|█████████▋| 221/228 [02:36<00:04,  1.43it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed

The Total Accuracy for Epoch 9: 92.16900633085604
Training Loss Epoch 9: 0.19574434336748692
Training F1-Score 9: 0.921245303870532


In [ ]:
print("*********************************VAL******************************")
y_val_true, y_val_pred = valid(model, VAL_LOADER, device, "Validation")
print("*********************************TEST******************************")
y_test_true, y_test_pred = valid(model, TEST_LOADER, device, "Test")

*********************************DEV******************************


  2%|▏         | 1/49 [00:00<00:15,  3.18it/s]


Validation F1-Score 9: 0.7260245475348451
-------------------------------------


100%|██████████| 49/49 [00:12<00:00,  3.93it/s]


The Total Accuracy for Epoch 9: 64.73649967469096
Validation F1-Score 9: 0.6425456936078208
*********************************TEST******************************


  2%|▏         | 1/49 [00:00<00:12,  3.92it/s]


Test F1-Score 9: 0.7822076978939724
-------------------------------------


 22%|██▏       | 11/49 [00:02<00:09,  3.84it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
100%|██████████| 49/49 [00:12<00:00,  3.79it/s]

The Total Accuracy for Epoch 9: 66.43222506393862
Test F1-Score 9: 0.6612468649906876


In [ ]:
results = {
        "val": evaluation_method(y_val_true, y_val_pred, IDS_TO_LABELS),
        "test": evaluation_method(y_test_true, y_test_pred, IDS_TO_LABELS)
    }
print(f"VAL, F1-Score: {results['val']['f1']}, Accuracy: {results['val']['accuracy']}")
print(f"TEST, F1-Score: {results['test']['f1']}, Accuracy: {results['test']['accuracy']}")

VAL, F1-Score: 0.6425456936078208, Accuracy: 0.6473649967469096
TEST, F1-Score: 0.6612468649906876, Accuracy: 0.6643222506393862
